In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/BTP-SEM-7/vqa/training")

In [1]:
import sys  
sys.path.append('../') 

In [3]:
from c3d.c3d import *
import imageio
from io import BytesIO
from keras.utils import to_categorical,Sequence
import numpy as np
import os
import parameters as params
from utils.array_util import *
import zipfile

In [4]:
def extract_preprocess_image(archive, filename):
    video = archive.read(filename)
    vid = imageio.get_reader(BytesIO(video), 'ffmpeg')
    frames=[]
    for num, image in enumerate(vid.iter_data()): 
        frames.append(image)
    # print("Shape of frames extracted", np.array(frames).shape)
    #Now shape of frames[] is [no of frames, 240, 320, 3]

    #Before doing c3d preprocessing, we need to split this into clips of 16 frames each
    clips = sliding_window(frames, params.frame_count, params.frame_count)
    # print("Number of clips after sliding window", len(clips))
    # print("Shape of each clip after sliding window", np.array(clips[0]).shape)
    #shape of clips is (noOfFrames/16, 16, 240, 320, 3) 

    #Now we need to c3d preprocess each clip
    preprocessed_clips=[]
    for i, clip in enumerate(clips):
        clip = np.array(clip)
        if len(clip) < params.frame_count:   #for ignoring the last clip with less than 16frames
            continue

        clip = preprocess_input(clip)
        preprocessed_clips.append(clip)
        # print("Pre-processed clip : ", i)

    # print("Shape after preprocessing", np.array(preprocessed_clips).shape)
    return np.array(preprocessed_clips)

In [5]:
#For Image preloaded image features
class My_Custom_Generator_Features(Sequence) :
    def __init__(self,filenames, labels, batch_size, archive) :
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.archive = archive
    def __len__(self) :
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx) :
        print("index:", idx)
        batch_x = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        #batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
        videos_x = np.empty((1,16,112,112,3))
        for counter, video in enumerate(batch_x):
            print("counter:", counter)
            op = extract_preprocess_image(self.archive, video)
            print(op.shape)
            videos_x = np.concatenate((videos_x, op), axis=0)
    
        batch_y = np.zeros((len(videos_x)-1, 4096))
        return videos_x[1:], batch_y

In [6]:
archive = zipfile.ZipFile('../../AnomalyVideos1.zip','r')

In [7]:
names = []
fileNames=[]
for name in archive.namelist():
    if name.endswith('.mp4'):
        fileNames.append(name)
        name = name.split('/')
        name = "/".join(name[1:])
        names.append(name)

In [9]:
videos_x = np.empty((1,16,112,112,3))
for counter, video in enumerate(fileNames[:5]):
    print("counter:", counter)
    op = extract_preprocess_image(archive, video)
    print(op.shape)
    videos_x = np.concatenate((videos_x, op), axis=0)
videos_x = videos_x[1:]

counter: 0
(86, 16, 112, 112, 3)
counter: 1
(112, 16, 112, 112, 3)
counter: 2
(320, 16, 112, 112, 3)
counter: 3
(170, 16, 112, 112, 3)
counter: 4
(213, 16, 112, 112, 3)


In [10]:
videos_x.shape

(901, 16, 112, 112, 3)

In [ ]:
preds = c3d.predict(videos_x)

In [ ]:
batches = int(np.ceil(len(fileNames)/batch_size))
predictions = np.empty((1,4096))
for i in range(0,batches):
    batch_x = fileNames[i*batch_size:(i+1)*batch_size]
    videos_x = np.empty((1,16,112,112,3))
    for counter, video in enumerate(batch_x):
        print("counter:", counter)
        op = extract_preprocess_image(archive, video)
        print(op.shape)
        videos_x = np.concatenate((videos_x, op), axis=0)
    videos_x = videos_x[1:]
    
    pred = model.predict(videos_x)
    predictions.concatenate((predictions,preds),axis=0)

predictions = predictions[1:]

In [13]:
def getFeatures(archive, batch_size=32):
    c3d = c3d_feature_extractor()
    names = []
    fileNames=[]
    for name in archive.namelist():
        if name.endswith('.mp4'):
            fileNames.append(name)
            name = name.split('/')
            name = "/".join(name[1:])
            names.append(name)
    
    num_clips_video = []
    fileNames = fileNames[:2]
    batches = int(np.ceil(len(fileNames)/batch_size))
    print("Batches:", batches)
    predictions = np.empty((1,4096))
    for i in range(0,batches):
        batch_x = fileNames[i*batch_size:(i+1)*batch_size]
        videos_x = np.empty((1,16,112,112,3))
        for counter, video in enumerate(batch_x):
            print("counter:", counter)
            op = extract_preprocess_image(archive, video)
            print(op.shape)
            num_clips_video.append(op.shape[0])
            videos_x = np.concatenate((videos_x, op), axis=0)
        videos_x = videos_x[1:]

        pred = c3d.predict(videos_x)
        predictions.concatenate((predictions,preds),axis=0)

    predictions = predictions[1:]
    
    return predictions, names, num_clips_video

In [ ]:
getFeatures(archive,5)

Batches: 1
counter: 0
(86, 16, 112, 112, 3)
counter: 1
(112, 16, 112, 112, 3)


In [ ]:
def getFeatures(archive, batch_size=32):
    c3d = c3d_feature_extractor()
    names = []
    fileNames=[]
    for name in archive.namelist():
        if name.endswith('.mp4'):
            fileNames.append(name)
            name = name.split('/')
            name = "/".join(name[1:])
            names.append(name)
    predictions = np.empty((1,4096))
    for counter, name in enumerate(fileNames):
        print("counter:",counter)
        op = extract_preprocess_image(archive, name)
        print(op.shape)
        pred = c3d.predict(op)
        predictions.concatenate((predictions,preds),axis=0)
    return predictions[1:]